In [1]:
pip install -r "requirement.txt"

Note: you may need to restart the kernel to use updated packages.


In [44]:
import sqlite3
import pandas as pd
db_file = "Fund.db"
tickers = ['AAPL','MSFT', 'GOOGL', 'AMZN', 'META', 'TSLA', 'NVDA', 'PYPL', 'ADBE', 'INTC',"BND", "AGG", "TLT", "GLD", "JNJ", "KO", "PG"]#tickers contenus dans MAIN_DATA_FRAME

start_date = "01/09/2022"#toute la periode start date (pour tout telecharger dun coup) + 3 mois avant
end_date = "31/12/2024"#pareil

## 1. Creation et peuplement de la base

In [45]:
from base_builder import DatabaseBuilder

In [46]:
db_manager = DatabaseBuilder(db_file)
db_manager.create_tables()
db_manager.insert_clients_data(12)
db_manager.insert_managers_data(3)

Tables created successfully.
12 clients insérés avec succès dans la table 'Clients'.
3 managers insérés avec succès dans la table 'Managers'.


In [47]:
db_manager.get_investment_amount_by_risk_type()
db_manager.insert_initial_cash_portfolios(start_date)


Résumé des investissements par risk type:
--------------------------------------------------
Risk Type: HY_EQUITY
Nombre de clients: 6
Montant total investi: 36,816,725.01 €
--------------------------------------------------
Risk Type: LOW_RISK
Nombre de clients: 12
Montant total investi: 50,206,035.78 €
--------------------------------------------------
Risk Type: LOW_TURNOVER
Nombre de clients: 18
Montant total investi: 67,733,394.74 €
--------------------------------------------------
Montants initiaux en cash insérés avec succès dans la table Portfolios.


## 2. Collecte et traitement des datas

In [48]:
from data_collector import GetData

In [35]:
gdata = GetData(tickers, start_date, end_date)
main_df =gdata.main_data_frame()  

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

# 3. Periode avant t = 0 (Periode de "test")

On peuple Products avec des trois mois de donnees avant la periode d'evaluation du fonds (qui commence le 01/01/2023)

In [36]:
from base_update import *
sub_start_date = "01/09/2022"#chaque semaine, les strat lancent la fct et definissent les sub périodes
#de cette maniere, products ne contient que la data jusqu'à la date t (no forward looking sur nos strats)
sub_end_date = "29/12/2022"

In [37]:
B_udt = BaseUpdate(tickers, sub_start_date, sub_end_date, db_file, main_df)
B_udt.update_products()

Successfully inserted 1411 new records into Products table for the period 01/09/2022 to 29/12/2022


### Initialisation des portefeuilles en fonction du cash initial

Pour la strategie HY equity, on prend les 5 tickers les plus performants de la periode (allant du 01/09/2022 au 30/12/2022)

In [38]:
B_udt.initialisation_portefeuille_HY()
update_portefeuille2(db_file,['BND', 'AGG', 'JNJ', 'KO', 'PG'], "LOW_RISK")

Le portefeuille HY_EQUITY est déjà initialisé
✅ Portefeuille LOW_RISK initialisé avec quantité = 0 pour tous les actifs.


In [ ]:
import sqlite3
import pandas as pd
import numpy as np
from datetime import datetime

def strategie_montecarlo_robuste(
    db_file: str,
    risk_type: str = "LOW_RISK",
    num_simulations: int = 200,
    num_days: int = 7,
    target_volatility: float = 0.10
):
    conn = sqlite3.connect(db_file)

    # 🔹 Récupérer automatiquement le MANAGER_ID
    manager_query = """
        SELECT MANAGER_ID FROM Portfolios
        WHERE RISK_TYPE = ? LIMIT 1
    """
    manager_row = pd.read_sql_query(manager_query, conn, params=(risk_type,))
    if manager_row.empty:
        return f"Aucun manager trouvé pour le portefeuille {risk_type}."
    manager_id = int(manager_row['MANAGER_ID'].iloc[0])

    # 🔹 Récupérer le budget disponible
    cash_query = """
        SELECT QUANTITY FROM Portfolios
        WHERE RISK_TYPE = ? AND TICKER = 'CASH'
    """
    cash_row = pd.read_sql_query(cash_query, conn, params=(risk_type,))
    if cash_row.empty:
        return "Pas de ligne 'CASH' dans le portefeuille."
    max_budget = float(cash_row['QUANTITY'].iloc[0])

    # 🔹 Récupérer les tickers présents dans le portefeuille
    initial_query = """
        SELECT TICKER, QUANTITY FROM Portfolios
        WHERE RISK_TYPE = ? AND TICKER != 'CASH'
    """
    initial_df = pd.read_sql_query(initial_query, conn, params=(risk_type,))
    portfolio_tickers = initial_df['TICKER'].tolist()

    if not portfolio_tickers:
        return "Aucun actif à simuler."

    # 🔹 Récupérer les données de prix
    price_data = {}
    for ticker in portfolio_tickers:
        df = pd.read_sql_query("""
            SELECT IMPORT_DATE, PRICE FROM Products
            WHERE TICKER = ? ORDER BY IMPORT_DATE ASC
        """, conn, params=(ticker,))
        if not df.empty:
            df['IMPORT_DATE'] = pd.to_datetime(df['IMPORT_DATE'], dayfirst=True)
            df.set_index('IMPORT_DATE', inplace=True)
            price_data[ticker] = df['PRICE']

    if not price_data:
        return "Données de prix manquantes."

    prices_df = pd.concat(price_data.values(), axis=1)
    prices_df.columns = list(price_data.keys())
    prices_df.dropna(inplace=True)

    log_returns = np.log(prices_df / prices_df.shift(1)).dropna()
    mean_returns = log_returns.mean()
    cov_matrix = log_returns.cov()
    latest_prices = prices_df.iloc[-1].values
    tickers = prices_df.columns.tolist()
    nb_assets = len(tickers)

    # 🔹 Quantités initiales
    initial_quantities = np.zeros(nb_assets)
    for i, ticker in enumerate(tickers):
        row = initial_df[initial_df['TICKER'] == ticker]
        if not row.empty:
            initial_quantities[i] = float(row['QUANTITY'].iloc[0])
    initial_cost = np.sum(initial_quantities * latest_prices)

    def simulate_gain(q):
        final_values = []
        for _ in range(30):
            future_prices = latest_prices.copy()
            for _ in range(num_days):
                daily_returns = np.random.multivariate_normal(mean_returns, cov_matrix)
                future_prices *= np.exp(daily_returns)
            final_values.append(np.sum(future_prices * q))
        return np.mean(final_values)

    initial_gain = simulate_gain(initial_quantities)
    initial_result = {
        'tickers': tickers,
        'quantities': dict(zip(tickers, initial_quantities.astype(int))),
        'volatility': np.sqrt(np.dot((initial_quantities / np.sum(initial_quantities)).T,
                                     np.dot(cov_matrix * num_days, (initial_quantities / np.sum(initial_quantities))))),
        'expected_return': np.sum(mean_returns * initial_quantities / np.sum(initial_quantities)) * num_days,
        'total_cost': initial_cost,
        'gain_or_loss': initial_gain - initial_cost,
        'performance': 'gain' if initial_gain - initial_cost > 0 else 'loss'
    }

    best_result = initial_result

    for _ in range(num_simulations):
        weights = np.random.random(nb_assets)
        weights /= np.sum(weights)
        candidate_quantities = np.floor((weights * max_budget) / latest_prices)

        # ✅ Contraintes de vente max 1000 et achat max 5000
        diff = candidate_quantities - initial_quantities
        if any(diff < -10000) or any(diff > 150000):
            continue

        port_vol = np.sqrt(np.dot((candidate_quantities / np.sum(candidate_quantities)).T,
                                  np.dot(cov_matrix * num_days, (candidate_quantities / np.sum(candidate_quantities)))))
        total_cost = np.sum(candidate_quantities * latest_prices)

        if port_vol <= target_volatility and total_cost <= max_budget:
            future_value = simulate_gain(candidate_quantities)
            gain_or_loss = future_value - total_cost

            if gain_or_loss > best_result['gain_or_loss']:
                best_result = {
                    'tickers': tickers,
                    'quantities': dict(zip(tickers, candidate_quantities.astype(int))),
                    'volatility': port_vol,
                    'expected_return': np.sum(mean_returns * candidate_quantities / np.sum(candidate_quantities)) * num_days,
                    'total_cost': total_cost,
                    'gain_or_loss': gain_or_loss,
                    'performance': 'gain' if gain_or_loss > 0 else 'loss'
                }

    # ✅ Si aucune stratégie trouvée, on conserve le portefeuille actuel
    if best_result == initial_result:
        conn.close()
        return "⚠️ Aucune stratégie optimale trouvée. Portefeuille conservé."

    # 🔹 Mise à jour Portfolios & Deals
    cursor = conn.cursor()
    cursor.execute("DELETE FROM Portfolios WHERE RISK_TYPE = ?", (risk_type,))
    conn.commit()

    for ticker in best_result['tickers']:
        quantity = int(best_result['quantities'][ticker])
        spot = float(prices_df[ticker].iloc[-1])
        cursor.execute("""
            INSERT INTO Portfolios (RISK_TYPE, TICKER, QUANTITY, MANAGER_ID, LAST_UPDATED, SPOT_PRICE)
            VALUES (?, ?, ?, ?, ?, ?)
        """, (risk_type, ticker, quantity, manager_id, datetime.now().strftime("%Y-%m-%d"), spot))

    cash_remaining = max_budget - best_result['total_cost']
    cursor.execute("""
        INSERT INTO Portfolios (RISK_TYPE, TICKER, QUANTITY, MANAGER_ID, LAST_UPDATED, SPOT_PRICE)
        VALUES (?, 'CASH', ?, ?, ?, 1)
    """, (risk_type, cash_remaining, manager_id, datetime.now().strftime("%Y-%m-%d")))

    # 🔹 Deals
    for i, ticker in enumerate(best_result['tickers']):
        old_qty = initial_quantities[i]
        new_qty = best_result['quantities'][ticker]
        delta = int(new_qty - old_qty)
        if delta != 0:
            trade_type = 'Buy' if delta > 0 else 'Sell'
            cursor.execute("""
                INSERT INTO Deals (RISK_TYPE, TICKER, MANAGER_ID, TRADE_TYPE, QUANTITY, BUY_PRICE)
                VALUES (?, ?, ?, ?, ?, ?)
            """, (
                risk_type,
                ticker,
                manager_id,
                trade_type,
                abs(delta),
                float(prices_df[ticker].iloc[-1])
            ))

    conn.commit()
    conn.close()
    return best_result



result = strategie_montecarlo_robuste("Fund.db", "LOW_RISK", 300 , 7, 0.1)

result

# 4. Execution des strategies

# Reconstruction de la composition des portefeuilles

# Visualisation des performances des portefeuilles